<a href="https://colab.research.google.com/github/jotisa/asistentecv/blob/main/CrearPineconeIndex_JoGeminiHelped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %% [markdown]
# # Crear Pinecone Index - Agenticus
# %%
#@title Haz click aquí -> X y presiona CTRL + ENTER

#Instalar paquetes
!pip install pdfplumber -q
!pip install pinecone-client -q
!pip install langchain -q
!pip install langchain-community -q
!pip install langchain_openai -q

# Importar paquetes
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
import os
from google.colab import drive
from google.colab import userdata
from tqdm import tqdm
import logging

# Logging setup
logging.basicConfig(level=logging.INFO)

#Setear variables
chunk_size = 1000
chunk_overlap = 200
splits = []
docs_usuario = userdata.get('DOCS_USUARIO')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_ENVIRONMENT'] = 'aws-starter'
embedding_model = OpenAIEmbeddings()

# Check if drive is already mounted
import os
if not os.path.exists('/content/drive'):
  #Montarse a Google Drive
  drive.mount('/content/drive')
else:
  print("Google Drive is already mounted.")

def load_and_split_pdf(file_path, chunk_size, chunk_overlap):
    try:
        loader = PDFPlumberLoader(file_path)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )
        return loader.load_and_split(text_splitter)
    except Exception as e:
        logging.error(f"Error loading {file_path}: {e}")
        return []

#  Jonatan Tisnado
# Convertir los PDF en un listado de langchain_core documents
for root, dirs, files in os.walk(docs_usuario):
    for file in tqdm(files, desc="Processing Files"):
        if file.lower().endswith('.pdf'):
            new_splits = load_and_split_pdf(os.path.join(root, file), chunk_size, chunk_overlap)
            splits.extend(new_splits)

#Crear retriever
try:
    vectorstore = Pinecone.from_documents(
        documents=splits, embedding=embedding_model, index_name='asistentecv'
    )
    print('El Pinecone index ha sido creado')
except Exception as e:
    logging.error(f"Error creating Pinecone index: {e}")